# Organize the code in Pytorch way

In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
# retina display
%config InlineBackend.figure_format = 'retina'

In [3]:
# write up your own neural network
class Linear: 

    def __init__(self, fan_in, fan_out, g, bias=True):
        self.weight = torch.randn(fan_in, fan_out, generator=g) / fan_in ** 0.5
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight, self.bias] if self.bias is not None else [self.weight]